In [1]:
#install PyKEEN
! python -c "import pykeen" || pip install git+https://github.com/pykeen/pykeen.git

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'pykeen'
  Running command git clone -q https://github.com/pykeen/pykeen.git 'C:\Users\Jeremy\AppData\Local\Temp\pip-req-build-akx0gui0'



  Cloning https://github.com/pykeen/pykeen.git to c:\users\jeremy\appdata\local\temp\pip-req-build-akx0gui0
  Created wheel for pykeen: filename=pykeen-1.5.1.dev0-py3-none-any.whl size=479570 sha256=6fe150da6e20742db0708ae26406f07a242c0c140036bb3c1209a79d98f713d4
  Stored in directory: C:\Users\Jeremy\AppData\Local\Temp\pip-ephem-wheel-cache-veh2fdam\wheels\24\dc\bd\878dc8b93f19515206086ec5d31314a24e994a2ebcc77e756d
  Created wheel for click-default-group: filename=click_default_group-1.2.2-py3-none-any.whl size=3389 sha256=9403916152e951bdda659252f9218afc84569b235b4049abb7434fd60ffe230c
  Stored in directory: c:\users\jeremy\appdata\local\pip\cache\wheels\7e\9f\ec\d087477ddc7c2807136ca08eb01b5f00e66da90eb4dbf323ab
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=722f8401142ca151c34834351fc8a81887ff4ef92999454d42e74fba77ec50d6
  Stored in directory: c:\users\jeremy\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da958

In [ ]:
#connect google drive to colab
from google.colab import drive
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [ ]:
#imports
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
from pykeen.sampling import NegativeSampler, BasicNegativeSampler
import torch
import numpy as np

In [ ]:
#paths to files
base_path = '/content/gdrive/MyDrive/datasets/kinship/'
sep = ','
dataset_name = 'kinship'
trainset_path = base_path + dataset_name + '_train.txt'
validset_path = base_path + dataset_name + '_valid.txt'
trainvalidset_path = base_path + dataset_name + '_train+valid.txt'

In [ ]:
#load triples from training+validation-set file
all_triples = TriplesFactory.from_path(trainvalidset_path)

In [ ]:
#load triples from training-set file and validation-set file
train_triples = TriplesFactory.from_path(trainset_path)
valid_triples = TriplesFactory.from_path(validset_path)

In [1]:
#perform a train-valid splitting
#trainset_percentage = 0.8
#train_triples, valid_triples = all_triples.split(ratios=trainset_percentage)

'\ntrainset_percentage = 0.8\ntrain_triples, valid_triples = all_triples.split(ratios=trainset_percentage)\n'

In [ ]:
#generate negative samples (if needed, e.g., for the downstream classifier)
#n_negatives_per_triple = 1
#ns_train = BasicNegativeSampler(triples_factory = train_triples, num_negs_per_pos = n_negatives_per_triple)
#negatives_train = ns_train.corrupt_batch(train_triples.mapped_triples)
#ns_validation = BasicNegativeSampler(triples_factory = validation_triples, num_negs_per_pos = n_negatives_per_triple)
#negatives_validation = ns_validation.corrupt_batch(validation_triples.mapped_triples)

In [ ]:
#train embeddings
result = pipeline( #https://pykeen.readthedocs.io/en/stable/api/pykeen.pipeline.pipeline.html#pykeen.pipeline.pipeline
    training=train_triples,
    testing=valid_triples,
    #model='TransR',
    model='TransE', #https://pykeen.readthedocs.io/en/latest/reference/models.html
                    #https://pykeen.readthedocs.io/en/latest/api/pykeen.models.Model.html#pykeen.models.Model
                    #https://pykeen.readthedocs.io/en/latest/api/pykeen.models.EntityRelationEmbeddingModel.html#pykeen.models.EntityRelationEmbeddingModel
    #loss='MSELoss', #default: the default loss specific of the model to be used (in most cases, 'MarginRankingLoss'); 
                    #https://pykeen.readthedocs.io/en/stable/reference/losses.html
    #model configuration
    model_kwargs = dict(
        #relation_dim=100, #dimensionality of the relation embeddings; default: 50; 
                          #in some models (like TransE) relation_dim cannot be set: it is assumed to be equal to embedding_dim
        embedding_dim=200 #dimensionality of the netity embeddings; default: 50
    ),
    loss_kwargs = dict(), #loss configuration
    training_loop='SLCWATrainingLoop', #default: 'SLCWATrainingLoop'; https://pykeen.readthedocs.io/en/stable/reference/training.html
    regularizer='LpRegularizer', #default: 'NoRegularizer'; https://pykeen.readthedocs.io/en/stable/reference/regularizers.html
    #regularizer configuration
    regularizer_kwargs=dict(
        p=2.0, #default: 2.0
        weight=1.0 #default: 1.0
    ),
    optimizer='SGD', #default: torch.optim.Adagrad; https://pytorch.org/docs/stable/optim.html%23torch.optim
    #optimizer configuration
    optimizer_kwargs = dict(
        lr=0.1
    ),
    negative_sampler='Basic', #default: 'Basic' (pykeen.sampling.BasicNegativeSampler); only allowed with training_loop='SLCWATrainingLoop'
    dimensions=200, #default: 50
    #training configuration
    training_kwargs=dict(
        num_epochs=200,
        use_tqdm_batch=False,
    ),  
    #runtime configuration
    random_seed=1234,
    device='gpu'#device='cpu'
)

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...


INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=2048.


INFO:pykeen.evaluation.evaluator:Evaluation took 0.10s seconds


In [ ]:
#save result to file (model and evaluation)
result_path = base_path + 'results'
result.save_to_directory(result_path)

In [ ]:
#extract entity embeddings
entity_ids = torch.LongTensor(range(all_triples.num_entities))
e_emb = result.model.entity_embeddings
e_emb.cpu()
e_emb_numpy = e_emb(entity_ids).detach().numpy()
entity2embedding = {}
for eid in range(all_triples.num_entities):
  e = all_triples.entity_id_to_label[eid]
  entity2embedding[e] = list(e_emb_numpy[eid])

In [ ]:
#extract relation embeddings
relation_ids = torch.LongTensor(range(all_triples.num_relations))
r_emb = result.model.relation_embeddings
r_emb.cpu()
r_emb_numpy = r_emb(relation_ids).detach().numpy()
relation2embedding = {}
for rid in range(all_triples.num_relations):
  r = all_triples.relation_id_to_label[rid]
  relation2embedding[r] = list(r_emb_numpy[rid])

In [ ]:
list(entity2embedding.items())[:2]

[('person0',
  [-0.03830957,
   -0.08314606,
   0.0031215334,
   -0.06076537,
   0.023113178,
   -0.0037718534,
   -0.039580446,
   0.035094153,
   0.08232855,
   0.024895491,
   -0.022422284,
   0.00033350496,
   0.06546119,
   -0.0789258,
   -0.057736225,
   -0.07081104,
   0.08317081,
   -0.02077934,
   0.085518956,
   0.007446033,
   0.07299697,
   -0.032261435,
   -0.08879858,
   -0.12501433,
   -0.016480062,
   -0.13691138,
   -0.09469248,
   -0.11105832,
   -0.02362788,
   0.03331221,
   0.06040625,
   0.027773472,
   -0.048045382,
   -0.075036086,
   -0.10255406,
   -0.010199388,
   0.059173916,
   -0.022227554,
   -0.06395364,
   -0.09868425,
   0.08284039,
   0.036163818,
   -0.036630407,
   -0.124933906,
   0.04558337,
   -0.028853204,
   -0.03240049,
   -0.05047469,
   0.16232257,
   0.032860104,
   -0.013319174,
   -0.07611109,
   -0.015186875,
   0.12280602,
   -0.034916952,
   -0.05807395,
   -0.07752786,
   0.07751399,
   0.038810894,
   -0.10198204,
   0.11525199,
   0

In [ ]:
list(relation2embedding.items())[:2]

[('term0',
  [0.07472012,
   0.055974208,
   0.15693758,
   0.026564203,
   -0.14677812,
   -0.13383926,
   0.06836072,
   0.11472272,
   -0.15255806,
   0.14218603,
   0.16492763,
   -0.030261371,
   0.026858222,
   -0.013954829,
   -0.1621713,
   0.09067295,
   0.1682137,
   -0.03713321,
   -0.055244107,
   0.13889055,
   -0.005233952,
   0.14609367,
   -0.11189553,
   -0.013490507,
   -0.07123864,
   -0.16055581,
   -0.05591469,
   -0.07736363,
   0.027290775,
   -0.16567566,
   -5.0418075e-05,
   -0.08495706,
   0.023583112,
   -0.13836183,
   -0.006488984,
   -0.015733976,
   0.09733157,
   0.15858504,
   0.05096898,
   0.0076322956,
   -0.07521578,
   0.005132796,
   -0.036174137,
   0.16300206,
   0.13805705,
   -0.065138005,
   0.009463025,
   0.122482255,
   -0.034432895,
   0.08056122,
   0.08358723,
   0.09368292,
   0.0021571976,
   0.13590764,
   0.10182261,
   0.160519,
   -0.06443603,
   -0.0329835,
   0.045950595,
   0.080460034,
   -0.17109272,
   0.14620975,
   0.0147

In [ ]:
#get scoring-function value of a knowledge-graph triple
h1,r1,t1 = 'person0', 'term0', 'person1'
h2,r2,t2 = 'person2', 'term1', 'person3'
h1id,r1id,t1id = all_triples.entity_to_id[h1], all_triples.relation_to_id[r1], all_triples.entity_to_id[t1]
h2id,r2id,t2id = all_triples.entity_to_id[h2], all_triples.relation_to_id[r2], all_triples.entity_to_id[t2]
triples_to_be_scored = torch.LongTensor([[h1id,r1id,t1id],[h2id,r2id,t2id]])
result.model.cpu()
scores = result.model.score_hrt(triples_to_be_scored)
scores_numpy = scores.detach().numpy()
scores_numpy

array([[-0.98509884],
       [-1.9159364 ]], dtype=float32)